In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import json
import numpy as np
import os

In [2]:
for mlp in [32, 64]:
    for gcn in [32, 64]:
        for cv in range(5):
            command = f"python train_gpr_gcn.py --exp=exp/MLP{mlp}_GCN{gcn}/CV{cv} --cv={cv} --cv_max=5 --model=cpr_gcn --mlp_hidden={mlp} --gcn_features={gcn} --epoch=101"
            print(command)

python train_gpr_gcn.py --exp=exp/MLP32_GCN32/CV0 --cv=0 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=32 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN32/CV1 --cv=1 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=32 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN32/CV2 --cv=2 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=32 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN32/CV3 --cv=3 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=32 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN32/CV4 --cv=4 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=32 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN64/CV0 --cv=0 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=64 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN64/CV1 --cv=1 --cv_max=5 --model=cpr_gcn --mlp_hidden=32 --gcn_features=64 --epoch=101
python train_gpr_gcn.py --exp=exp/MLP32_GCN64/CV2 --cv=2 --cv_max=5 --model=cpr_gcn --mlp_

In [3]:
exp_root = "exp"
result_df = pd.DataFrame(columns=["mlp", "gcn", "cv", "acc", "precision", "recall", "f1"])

for mlp in [64, 128]:
    for gcn in [128, 256]:
        for cv in range(5):
            exp_path = os.path.join(exp_root, f"MLP{mlp}_GCN{gcn}/CV{cv}")
            df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
            max_row = df['acc'].argmax()
            result_df = result_df.append({"mlp": mlp, "gcn": gcn, "cv": cv,
                                          "acc": df['acc'].loc[max_row],
                                          "precision": df['precision'].loc[max_row],
                                          "recall": df['recall'].loc[max_row],
                                          "f1": df['f1'].loc[max_row]}, ignore_index=True)
            

In [4]:
result_df

,mlp,gcn,cv,acc,precision,recall,f1
0,64.0,128.0,0.0,0.5274,0.6118,0.5274,0.4984
1,64.0,128.0,1.0,0.5011,0.4531,0.5011,0.4266
2,64.0,128.0,2.0,0.4651,0.4942,0.4651,0.4721
3,64.0,128.0,3.0,0.3815,0.2927,0.3815,0.311
4,64.0,128.0,4.0,0.4156,0.3798,0.4156,0.3878
5,64.0,256.0,0.0,0.4814,0.416,0.4814,0.4416
6,64.0,256.0,1.0,0.3036,0.1111,0.3036,0.1468
7,64.0,256.0,2.0,0.4148,0.3288,0.4148,0.3421
8,64.0,256.0,3.0,0.4224,0.3077,0.4224,0.3506
9,64.0,256.0,4.0,0.2978,0.0887,0.2978,0.1367


In [6]:
result_df.groupby(["mlp", "gcn"]).mean()

cv      acc  precision   recall       f1
mlp   gcn                                             
64.0  128.0  2.0  0.45814    0.44632  0.45814  0.41918
      256.0  2.0  0.38400    0.25046  0.38400  0.28356
128.0 128.0  2.0  0.42106    0.34914  0.42106  0.35548
      256.0  2.0  0.44194    0.35544  0.44194  0.38284

In [10]:
# BEST parameters
mlp=64
gcn=128
branches = ["LMA", "LAD", "LCX", "D", "OM", "weighted avg"]

acc, prec, rec, f1 = {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}

for cv in range(5):
    exp_path = os.path.join(exp_root, f"MLP{mlp}_GCN{gcn}/CV{cv}")
    df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
    max_row = df['acc'].argmax()
    epoch = df['epoch'].iloc[max_row]
    #  print(epoch)

    clf_report = json.load(open(os.path.join(exp_path, f"clf_report_{epoch:04d}.json"), "r"))
    cm = np.load(os.path.join(exp_path, f"confusion_matrix_{epoch:04d}.npy"))
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # print(cm.diagonal())
    for idx, k in enumerate(branches):
        if idx < len(branches) - 1:
            acc[k].append(cm.diagonal()[idx])
        else:
            acc[k].append(clf_report["accuracy"])
        prec[k].append(clf_report[k]['precision'])
        rec[k].append(clf_report[k]['recall'])
        f1[k].append(clf_report[k]['f1-score'])

table_df = pd.DataFrame(columns=["metric"]+branches)
row_data = {"metric": "acc"}
for k in branches:
    row_data.update({k: f"{np.mean(acc[k]):0.4f}±{np.std(acc[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "pre"}
for k in branches:
    row_data.update({k: f"{np.mean(prec[k]):0.4f}±{np.std(prec[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "rec"}
for k in branches:
    row_data.update({k: f"{np.mean(rec[k]):0.4f}±{np.std(rec[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "f1"}
for k in branches:
    row_data.update({k: f"{np.mean(f1[k]):0.4f}±{np.std(f1[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
table_df.to_csv("cpr_gcn.csv", index=False)

80
20
30
60
20
